# L8b: Single Index Model Portfolio Optimization
In this lecture, we will explore the Single Index Model (SIM) and its application in portfolio optimization. The SIM simplifies the analysis of asset returns by assuming that the returns of individual assets are influenced by a single common factor, typically the market return. Here, we'll look at using single index models to describe portfolios composed of only risky assets and a combination of risky and risk-free assets.

> __Learning Objectives:__
> 
> By the end of this lecture, you will be able to:
> * __SIM portfolio risk and return:__ Compute portfolio expected return and variance using single index model parameters, reducing estimation from $N(N+3)/2$ to $2N+2$ parameters.
> * __SIM covariance construction:__ Build covariance matrices where all asset correlations arise from common market exposure through $\beta_i\beta_j\sigma_m^2$.
> * __Capital allocation and the Separation Theorem:__ Construct optimal portfolios by combining the tangent portfolio (which maximizes the Sharpe ratio) with a risk-free asset.

Let's get started!
___

## Examples

Today, we will be using the following examples to illustrate key concepts:

> [▶ Let's estimate single index model parameter uncertainty from historical data](CHEME-5660-L7a-Example-SIM-Parameter-Uncertainty-Fall-2025.ipynb). In this example, we estimate single index models from historical data and quantify the uncertainty in our parameter estimates using bootstrap simulation. Bootstrapping allows us to assess the variability of our estimates and construct confidence intervals, providing insights into the reliability of our model parameters, using simulation techniques.

> [▶ Let's estimate a portfolio of risky assets using single index models](CHEME-5660-L8b-Example-SIM-MinVar-RA-Fall-2025.ipynb). In this example, we'll construct a risky asset portfolio using the single index model. We'll compare the performance of this SIM-based portfolio to one optimized using traditional methods based on historical data. This comparison will help us understand the advantages and limitations of using the single index model for portfolio optimization.

> [▶ Let's build a portfolio with risky and risk-free assets](CHEME-5660-L8b-Example-SIM-MinVar-RRFA-Fall-2025.ipynb). In this example, we construct a portfolio that includes both risky assets and a risk-free asset, such as a treasury STRIPs bond. We will use the single index model to optimize the portfolio, aiming to minimize risk while achieving a specified return. We'll explore how the inclusion of a risk-free asset affects the portfolio's risk-return profile and think about what this means for real-world investing.
___

## Concept Review: Single Index Model (SIM)
The Single Index Model (SIM) is a simplified approach to understanding the relationship between the returns of individual assets and the overall market. It assumes that the return of an asset can be expressed as a linear function of the market return plus a unique component specific to the asset.


> __Single Index Model (SIM)__
> 
> Single index models are factor models that consider only the return (growth) of the market factor. These models were originally developed by Sharpe, 1963: [Sharpe, William F. (1963). "A Simplified Model for Portfolio Analysis". Management Science, 9(2): 277-293. doi:10.1287/mnsc.9.2.277.](https://pubsonline.informs.org/doi/abs/10.1287/mnsc.9.2.277)
>
> Suppose the growth of firm $i$ at time $t$ is denoted by $g^{(t)}_{i}$. Then, the single index model of the return (growth rate) is given by:
> $$
g^{(t)}_{i} = \alpha_{i} + \beta_{i}\;g^{(t)}_{M} + \epsilon^{(t)}_{i}
$$
> where $\alpha_{i}$ is the _idiosyncratic (firm-specific) growth_, $\beta_{i}$ is the component of the growth rate of firm $i$ explained by the market (it is also a measure of risk), $g^{(t)}_{M}$ is the growth rate of the market index at time $t$, and $\epsilon^{(t)}_{i}$ denotes an error model associated with firm $i$ (describes growth rate not captured by the firm or market factors). 



### Simulation: Bootstrapping parameter uncertainty
Finally, an interesting way to explore parameter uncertainty is through a Monte Carlo simulation procedure called bootstrapping.

> __Bootstrapping__ The idea is to use our estimated parameters and error model to generate many __synthetic datasets__, then examine the empirical distribution of parameter estimates that we obtain from re-estimating parameters using these datasets. This approach is called __bootstrapping__ because we are _pulling ourselves up by our bootstraps_, i.e., using our fitted model to generate new data and re-estimate parameters, thereby empirically exploring the sampling distribution of our estimator. 

This allows us to see if the empirical results match our theoretical expectations. Let's look at some pseudocode for how we might implement this simulation.

__Initialization:__ Given the design matrix $\hat{\mathbf{X}}$, the estimated parameters $\hat{\boldsymbol{\theta}}$ and the error variance $\hat{\sigma}_{\epsilon}^2$ from our observed data, a value for the regularization parameter $\delta\geq{0}$, and the number of samples to generate $K$. 

For each $k = 1, 2, \ldots, K$: __do__:
1. Generate synthetic errors: $\boldsymbol{\varepsilon}^{(k)} \sim \mathcal{N}(\mathbf{0}, \Delta{t}\;\hat{\sigma}_{\epsilon}^2\;\mathbf{I})$
2. Create synthetic observations: $\mathbf{y}^{(k)} \gets \hat{\mathbf{X}}\;\hat{\boldsymbol{\theta}} + \boldsymbol{\varepsilon}^{(k)}$
3. Estimate parameters from the synthetic observation: $\hat{\boldsymbol{\theta}}^{(k)} \gets \left(\hat{\mathbf{X}}^{\top}\hat{\mathbf{X}} + \delta\;\mathbf{I}\right)^{-1}\hat{\mathbf{X}}^{\top}\mathbf{y}^{(k)}$

Analyze the empirical distribution of $\left\{\hat{\boldsymbol{\theta}}^{(1)}, \hat{\boldsymbol{\theta}}^{(2)}, \ldots, \hat{\boldsymbol{\theta}}^{(K)}\right\}$. 

The empirical mean and covariance of the simulated parameter estimates should approximate our theoretical result:
$$
\begin{align*}
\text{Empirical mean} &\approx \mathbf{S}\;\hat{\boldsymbol{\theta}}\\
\text{Empirical covariance} &\approx \Delta{t}\;\hat{\sigma}_{\epsilon}^2\;\left(\hat{\mathbf{X}}^{\top}\hat{\mathbf{X}} + \delta\;\mathbf{I}\right)^{-1}
\end{align*}
$$


Let's look at an example where we quantify parameter uncertainty using bootstrap simulation.

> __Example__
>
> [▶ Let's estimate single index model parameter uncertainty from historical data](CHEME-5660-L7a-Example-SIM-Parameter-Uncertainty-Fall-2025.ipynb). In this example, we estimate single index models from historical data and quantify the uncertainty in our parameter estimates using bootstrap simulation. Bootstrapping allows us to assess the variability of our estimates and construct confidence intervals, providing insights into the reliability of our model parameters, using simulation techniques.

___

## Risk and Return Measure using Single Index Models
Using the single index model, we can derive expressions for the expected return and risk (variance) of a portfolio composed of multiple assets. Thus, we have a model for these quantities that does not require us to estimate all pairwise covariances directly from data.

> __Why use Single Index Models?__ 
> 
> The SIM dramatically reduces the number of parameters that must be estimated for portfolio optimization. 
> * For a traditional portfolio of $N$ assets, the traditional approach requires estimating $N$ expected returns and $N(N+1)/2$ unique covariances, a total of $N(N+3)/2$ parameters. 
> * A SIM portfolio requires only $2N$ parameters ($\alpha_i$ and $\beta_i$ for each asset) plus two market parameters ($\mathbb{E}[g_M]$ and $\sigma_m^2$), totaling $2N+2$ parameters. 
> 
> For large portfolios, this reduction is substantial: a 100-asset portfolio requires 5,150 traditional parameters versus only 202 SIM parameters. This parsimony reduces estimation error and improves out-of-sample portfolio performance.

### Reward Measure
Suppose we have a portfolio $\mathcal{P}$ composed of $N$ assets, with weights (dollar fraction) $w_i$ for each asset $i$. The expected annualized growth rate (return) of the portfolio (reward measure) can be expressed as the weighted sum of the expected returns of the individual assets:
$$
\begin{align*}
\mathbb{E}[g_{\mathcal{P}}] &= \sum_{i=1}^{N} w_i\;\mathbb{E}[g_i]\\
&= \sum_{i=1}^{N} w_i\;\left(\alpha_i + \beta_i\;\mathbb{E}[g_M]\right)\\
&= \underbrace{\sum_{i=1}^{N} w_i\;\alpha_i}_{\;\alpha_{\mathcal{P}}} + \underbrace{\left(\sum_{i=1}^{N} w_i\;\beta_i\right)}_{\;\beta_{\mathcal{P}}}\;\mathbb{E}[g_M]\\
&= \alpha_{\mathcal{P}} + \beta_{\mathcal{P}}\;\mathbb{E}[g_M]\quad\blacksquare
\end{align*}
$$

### Risk Measure
The risk of a portfolio is typically measured by the variance of its growth rate (returns). 
The variance of the growth rate (return) of portfolio $\mathcal{P}$ is given by:
$$
\begin{align*}
\operatorname{Var}(g_{\mathcal{P}}) &= 
\sum_{i\in\mathcal{P}}\sum_{j\in\mathcal{P}}w_{i}w_{j}
\operatorname{Cov}\left(g_{i},g_{j}\right)
\end{align*}
$$
where $w_{\star}$ denotes the fraction of asset $\star$ in the portfolio $\mathcal{P}$, and 
$\operatorname{Cov}\left(g_{i},g_{j}\right)$ denotes the covariance between the growth rate of assets $i$ and $j$. Substituting the single index model for the growth rate of assets $i$ and $j$ into the covariance expression (and simplifying) gives:
$$
\begin{align*}
\operatorname{Cov}(g_{i}, g_{j}) = \begin{cases}
\beta_{i}^{2}\sigma_{m}^{2}+\Delta{t}\;\sigma_{\epsilon_{i}}^{2} & i = j \\
\beta_{i}\beta_{j}\sigma_{m}^2 & i \neq j
\end{cases}
\end{align*}
$$
where $\sigma_{m}^2$ denotes the variance of the growth rate (return) of the market factor, 
$\sigma_{\epsilon_{i}}^{2}$ denotes the variance of the firm-specific error model, and $\Delta{t}$ is the time step (e.g., $\Delta{t} = 1/252$ for daily data annualized).

#### Error model variance estimation
We assumed the error model $\boldsymbol{\varepsilon} \sim \mathcal{N}(\mathbf{0},\Delta{t}\;\sigma_{\epsilon}^{2}\;\mathbf{I})$, where $\sigma_{\epsilon}^{2}$ is the variance of the error model. Of course, we don't know $\sigma_{\epsilon}^{2}$, so we need to estimate it from the data. A common approach is to estimate the variance of the residuals:
$$
\begin{align*}
\hat{\sigma}_{\epsilon_i}^{2} &= \frac{1}{\Delta{t}(n-p)}\;\lVert~\underbrace{\mathbf{y}_{i} - \hat{\mathbf{X}}\;\hat{\boldsymbol{\theta}_{i}}}_{\text{residual}\;\mathbf{r}}~\rVert^{2}_{2}
\end{align*}
$$
where $n$ is the number of training examples, $p = 2$ is the number of model parameters (including the intercept), $\mathbf{y}_{i}$ is the observed growth rate for firm $i$, and $\hat{\boldsymbol{\theta}}_{i}$ is the estimated parameter vector (the $\alpha_{i}$ and $\beta_{i}$ values for asset $i$).

__Advanced:__ A detailed [derivation of the covariance expression is provided here!](CHEME-5660-L8b-Derivation-SIM-Covariance-Fall-2025.ipynb).
___

### Constructing the SIM Covariance Matrix

Now that we understand the theory, let's see how to actually construct a covariance matrix using single index model parameters. This process is much simpler than estimating the full covariance matrix from historical data.

Let's look at some pseudocode for SIM covariance matrix construction.

__Initialize:__ Given a portfolio with $N$ assets, single index model parameters $\alpha_i$, $\beta_i$ for each asset $i = 1, \ldots, N$, market variance $\sigma_m^2$, and residual variances $\sigma_{\epsilon_i}^2$ for each asset $i$.
Initialize the $N \times N$ covariance matrix $\boldsymbol{\Sigma}$.

For $i = 1$ to $N$ __do:__
1. For $j = 1$ to $N$ __do:__
    * __if__ $i = j$: Compute diagonal element: $\boldsymbol{\Sigma}_{i,j} \gets \beta_i^2 \sigma_m^2 + \Delta{t}\;\sigma_{\epsilon_i}^2$
    * __else:__ Compute off-diagonal element: $\boldsymbol{\Sigma}_{i,j} \gets \beta_i \beta_j \sigma_m^2$

Return the covariance matrix $\boldsymbol{\Sigma}$.

The SIM covariance formula reveals important economic insights about asset relationships:

$$\operatorname{Cov}(g_i, g_j) = \beta_i \beta_j \sigma_m^2 \quad \text{for } i \neq j$$

> __What does this mean?__
>
> * __Market-driven correlations:__ The covariance between any two assets $i$ and $j$ is entirely driven by their common exposure to market movements. The product $\beta_i \beta_j$ captures how both assets respond to the same market factor.
>
> * __No direct firm-to-firm relationships:__ The SIM assumes that after accounting for market effects, asset returns are independent. For example, if Apple and Microsoft both have $\beta \approx 1.2$, their covariance is approximately $1.2 \times 1.2 \times \sigma_m^2 = 1.44\sigma_m^2$, regardless of any direct business relationships between the companies.
>
> * __Beta as a correlation amplifier:__ High-beta stocks ($\beta > 1$) will have stronger covariances with other high-beta stocks. Two stocks with $\beta = 1.5$ will have covariance $2.25\sigma_m^2$, while two stocks with $\beta = 0.5$ will have covariance only $0.25\sigma_m^2$.
>
> * __Interpretation of diagonal elements:__ For $i = j$, we get $\operatorname{Var}(g_i) = \beta_i^2 \sigma_m^2 + \Delta{t}\;\sigma_{\epsilon_i}^2$. This decomposes total risk into:
>   - **Systematic risk** ($\beta_i^2 \sigma_m^2$): Risk from market movements that affects all assets
>   - **Idiosyncratic risk** ($\Delta{t}\;\sigma_{\epsilon_i}^2$): Firm-specific risk independent of market movements
>
> * __When SIM works well:__ The model is most accurate when asset returns are primarily driven by a common market factor (e.g., S&P 500 index funds, large-cap tech stocks). It works less well when significant correlations exist outside of market effects (e.g., two companies in a merger negotiation, or gold mining stocks with gold prices as a dominant factor beyond the general market).

__Example:__ Consider three assets with $\beta_1 = 0.8$, $\beta_2 = 1.2$, $\beta_3 = 1.5$, and $\sigma_m^2 = 0.04$:
- $\operatorname{Cov}(g_1, g_2) = 0.8 \times 1.2 \times 0.04 = 0.0384$
- $\operatorname{Cov}(g_1, g_3) = 0.8 \times 1.5 \times 0.04 = 0.0480$
- $\operatorname{Cov}(g_2, g_3) = 1.2 \times 1.5 \times 0.04 = 0.0720$

Notice that assets 2 and 3 (both high-beta) have the strongest covariance, even if they're in completely different industries! This is the key assumption (and potential limitation) of the single index model.
___

## Risky Asset Portfolios using Single Index Models
The risky portfolio allocation problem for a portfolio $\mathcal{P}$ using the single index model is:
$$
\begin{align*}
\text{minimize}~\operatorname{Var}(g_{\mathcal{P}}) &= \sum_{i\in\mathcal{P}}\sum_{j\in\mathcal{P}}w_{i}w_{j}
\operatorname{Cov}\left(g_{i},g_{j}\right) \\
\text{subject to}~\mathbb{E}[g_{\mathcal{P}}]& = \alpha_{\mathcal{P}}+\beta_{\mathcal{P}}\;\mathbb{E}[g_{M}] = {R^{*}} \\
\alpha_{\mathcal{P}} & = \sum_{i\in\mathcal{P}}w_{i}\;\alpha_{i}\\
\beta_{\mathcal{P}} & = \sum_{i\in\mathcal{P}}w_{i}\;\beta_{i} \\
\sum_{i\in\mathcal{P}}w_{i} & = 1 \\
w_{i}&\geq 0 \qquad \forall{i}\in\mathcal{P} \\
\end{align*}
$$
where the covariance between assets $i$ and $j$ is given by:
$$
\begin{align*}
\operatorname{Cov}(g_{i}, g_{j}) & = \begin{cases}
\beta_{i}^{2}\sigma_{m}^{2}+\Delta{t}\;\sigma_{\epsilon_{i}}^{2} & i = j \\
\beta_{i}\beta_{j}\sigma_{m}^2 & i \neq j
\end{cases} \\
\end{align*}
$$

Let's look at an example of optimizing a risky asset portfolio using the single index model, and compare it to the same assets optimized using the full covariance matrix and expected returns estimated from historical data.

> __Example__
>
> [▶ Let's estimate a portfolio of risky assets using single index models](CHEME-5660-L8b-Example-SIM-MinVar-RA-Fall-2025.ipynb). In this example, we'll construct a risky asset portfolio using the single index model. We'll compare the performance of this SIM-based portfolio to one optimized using traditional methods based on historical data. This comparison will help us understand the advantages and limitations of using the single index model for portfolio optimization.

___

<div>
    <center>
        <img src="figs/Fig-MinVar-RiskFree-Schematic.svg" width="680"/>
    </center>
</div>

## Risky and Risk-Free Asset Portfolios using Single Index Models
We can also consider portfolios that include both risky assets and a risk-free asset. The risk-free asset has a known growth rate (return) $g_f$ and zero variance. For our example, we'll assume a zero-coupon bond, e.g., a STRIPS bond, that matures in some specified time horizon.

The minimum variance portfolio problem for a portfolio $\mathcal{P}$ using the single index model with a combination of risky and risk-free assets is given by:
$$
\begin{align*}
\text{minimize}~\operatorname{Var}(g_{\mathcal{P}}) &= \sum_{i\in\mathcal{P}}\sum_{j\in\mathcal{P}}w_{i}w_{j}
\operatorname{Cov}\left(g_{i},g_{j}\right) \\
\text{subject to}~\mathbb{E}[g_{\mathcal{P}}]& = w_{f}g_{f}+\alpha_{\mathcal{P}}+\beta_{\mathcal{P}}\;\mathbb{E}[g_{M}] = {R^{*}}\\
\alpha_{\mathcal{P}} & = \sum_{i\in\mathcal{P}}w_{i}\;\alpha_{i}\\
\beta_{\mathcal{P}} & = \sum_{i\in\mathcal{P}}w_{i}\;\beta_{i} \\
w_{f}+\sum_{i\in\mathcal{P}}w_{i} & = 1 \\
w_{f}&\geq{0}\\
w_{i}&\geq{0}\qquad{\forall{i}\in\mathcal{P}}
\end{align*}
$$
where the covariance between assets $i$ and $j$ is given by:
$$
\begin{align*}
\operatorname{Cov}(g_{i}, g_{j}) & = \begin{cases}
\beta_{i}^{2}\sigma_{m}^{2}+\Delta{t}\;\sigma_{\epsilon_{i}}^{2} & i = j \\
\beta_{i}\beta_{j}\sigma_{m}^2 & i \neq j
\end{cases} \\
\end{align*}
$$

The terms $w_{i}\geq{0}$ denote the fraction of risky asset $i\in\mathcal{P}$, 
the quantity $w_{f}$ denotes the fraction of risk-free assets in the portfolio, 
$g_{f}$ denotes the risk-free rate or return, and $R^{*}$ is the minimum required growth rate (return) 
for the overall portfolio $\mathcal{P}$.
___

### The Capital Allocation Line and Tangent Portfolio

When we introduce a risk-free asset into our portfolio optimization problem, something remarkable happens: the efficient frontier transforms into a straight line in risk-return space. This line is called the __Capital Allocation Line (CAL)__.

The Capital Allocation Line describes all portfolios that can be formed by combining a risk-free asset (e.g., Treasury STRIPS) with return $g_f$ and zero variance, and a single optimal risky portfolio called the __tangent portfolio__ (denoted by $T$). The tangent portfolio has expected return $\mathbb{E}[g_T]$ and variance $\sigma_T^2$. Any portfolio on the CAL can be expressed as:
$$
\begin{align*}
\mathbb{E}[g_{\mathcal{P}}] &= w_f g_f + (1 - w_f)\;\mathbb{E}[g_T]\\
\sigma_{\mathcal{P}} &= (1 - w_f) \sigma_T
\end{align*}
$$
where $w_f$ is the fraction invested in the risk-free asset, $(1-w_f)$ is the fraction invested in the tangent portfolio, $\sigma_{\mathcal{P}} = \sqrt{\operatorname{Var}(g_{\mathcal{P}})}$ is the standard deviation of the portfolio return, and $\sigma_T = \sqrt{\operatorname{Var}(g_T)}$ is the standard deviation of the tangent portfolio return. To derive the CAL equation, we solve for $w_f$ from the second equation and substitute into the first:
$$
\begin{align*}
\sigma_{\mathcal{P}} &= (1 - w_f) \sigma_T\\
\frac{\sigma_{\mathcal{P}}}{\sigma_T} &= 1 - w_f\\
w_f &= 1 - \frac{\sigma_{\mathcal{P}}}{\sigma_T}
\end{align*}
$$
Substituting this expression for $w_f$ into the expected return equation:
$$
\begin{align*}
\mathbb{E}[g_{\mathcal{P}}] &= w_f g_f + (1 - w_f) \mathbb{E}[g_T]\\
&= \left(1 - \frac{\sigma_{\mathcal{P}}}{\sigma_T}\right) g_f + \frac{\sigma_{\mathcal{P}}}{\sigma_T} \mathbb{E}[g_T]\\
&= g_f - \frac{\sigma_{\mathcal{P}}}{\sigma_T} g_f + \frac{\sigma_{\mathcal{P}}}{\sigma_T} \mathbb{E}[g_T]\\
&= g_f + \frac{\sigma_{\mathcal{P}}}{\sigma_T} \left(\mathbb{E}[g_T] - g_f\right)\\
&= g_f + \underbrace{\frac{\mathbb{E}[g_T] - g_f}{\sigma_T}}_{\text{Sharpe ratio T.P.}}\;\sigma_{\mathcal{P}}\quad\blacksquare
\end{align*}
$$
This is a __linear relationship__ between expected return and risk. The slope of this line is the __Sharpe ratio of the tangent portfolio__, which measures the excess return per unit of risk for that optimal risky portfolio.

To find the tangent portfolio, we identify the risky portfolio that maximizes the Sharpe ratio. Geometrically, it's the point where a line from the risk-free rate is tangent to the risky-only efficient frontier. Using the single index model, we solve:
$$
\begin{align*}
\text{maximize} &\quad \frac{\mathbb{E}[g_{\mathcal{P}}] - g_f}{\sigma_{\mathcal{P}}} = \frac{\alpha_{\mathcal{P}} + \beta_{\mathcal{P}}\;\mathbb{E}[g_M] - g_f}{\sigma_{\mathcal{P}}}\\
\text{subject to} &\quad \sum_{i\in\mathcal{P}}w_{i} = 1\\
&\quad w_{i} \geq 0 \qquad \forall{i}\in\mathcal{P}
\end{align*}
$$
The portfolio $\mathcal{P}$ that solves this optimization problem is the tangent portfolio $T$. Once found, its Sharpe ratio $\frac{\mathbb{E}[g_T] - g_f}{\sigma_T}$ becomes the slope of the CAL.

Alternatively (and equivalently), we can find it by solving the minimum variance problem at a high enough return level such that $w_f = 0$ (i.e., the portfolio lies on the risky-only efficient frontier). 

Geometrically, the tangent portfolio is the unique point where the CAL touches the efficient frontier, it's the portfolio that allows us to achieve the steepest possible CAL slope starting from the risk-free rate.

> __The Separation Theorem__
>
> Every investor, regardless of their risk preferences, should hold the same risky portfolio, i.e., the tangent portfolio. What differs between investors is how much they allocate between this tangent portfolio and the risk-free asset. Risk-averse investors hold more of the risk-free asset, while risk-seeking investors may borrow money (making $w_f$ negative) to invest more than 100% of their wealth in the tangent portfolio.
>
> This result is known as the __Separation Theorem__ or __Two-Fund Separation Theorem__, originally developed by James Tobin in his 1958 paper on liquidity preference. [Tobin, James (1958). "Liquidity Preference as Behavior Towards Risk". The Review of Economic Studies, 25(2): 65-86.](https://doi.org/10.2307/2296205) The investment decision (which risky portfolio to hold) is separated from the financing decision (how much to borrow or lend at the risk-free rate).

> __Practical Note: Concentration Constraints__
>
> In practice, unconstrained tangent portfolio optimization often produces extreme weights in a few assets, especially with imprecise estimates. Practitioners commonly impose concentration constraints like $0 \leq w_i \leq u$ to force diversification. While these constraints theoretically reduce the Sharpe ratio, they often improve out-of-sample performance by making portfolios more robust to estimation error.

Let's look at an example that demonstrates these concepts by constructing the Capital Allocation Line and finding the tangent portfolio.

> __Example__
>
> [▶ Let's build a portfolio with risky and risk-free assets](CHEME-5660-L8b-Example-SIM-MinVar-RRFA-Fall-2025.ipynb). In this example, we construct a portfolio that includes both risky assets and a risk-free asset, such as a treasury STRIPs bond. We will use the single index model to optimize the portfolio, aiming to minimize risk while achieving a specified return. We'll explore how the inclusion of a risk-free asset affects the portfolio's risk-return profile, construct the Capital Allocation Line, and identify the tangent portfolio.

___

## Summary
In this lecture, we applied the single index model to portfolio optimization, demonstrating how SIM simplifies the multiasset allocation problem while preserving essential risk-return relationships.

> __Key Takeaways:__
> 
> * __SIM reduces estimation complexity:__ The single index model requires only $2N+2$ parameters (alpha and beta for each asset, plus market mean and variance) instead of $N(N+3)/2$ parameters for traditional portfolio optimization, dramatically reducing estimation error for large portfolios.
> * __SIM covariances are market-driven:__ Asset covariances are determined entirely by their market exposures through $\beta_i\beta_j\sigma_m^2$, meaning all correlations arise from common market movements rather than direct firm-to-firm relationships.
> * __The Separation Theorem simplifies investing:__ When a risk-free asset is available, all investors should hold the same risky portfolio (the tangent portfolio, which maximizes the Sharpe ratio) and differ only in how much they allocate between this portfolio and the risk-free asset.

In the next lecture, we will explore advanced portfolio reallocation strategies.
___

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.

___